In [49]:
import numpy as np
import math

In [23]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
import tensorflow as tf

In [8]:
paths = [str(x) for x in Path('../prompts_dataset/').glob('*.txt')]

# Tokenizer

In [10]:
tokenizer = ByteLevelBPETokenizer()

In [11]:
tokenizer.train(files=paths, vocab_size=30_522, min_frequency=2, special_tokens= [
    '<s>', '</s>', '<pad>', '<unk>', '<mask'
])

In [12]:
tokenizer.save_model('neublla_codex')

['neublla_codex\\vocab.json', 'neublla_codex\\merges.txt']

Vocab size

In [26]:
tokenizer.get_vocab_size()

2315

In [27]:
token_embed = tf.keras.layers.Embedding(tokenizer.get_vocab_size(), 4)

one input sample text preprocessing steps

In [30]:
sample_text = "Open Word and write in it summerization about palastine"

sample_text_encode = tokenizer.encode(sample_text)
sample_text_tokens = sample_text_encode.tokens
sample_text_tokens_ids = sample_text_encode.ids
sample_text_tokens_seq = np.array(sample_text_tokens_ids)

In [32]:
print("Sample text encoding info")
print(sample_text_encode)
print("Sample text tokens")
print(sample_text_tokens)
print("Sample text tokens ids")
print(sample_text_tokens_ids)
print("Sample text tokens seq")
print(sample_text_tokens_seq)

Sample text encoding info
Encoding(num_tokens=15, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Sample text tokens
['Open', 'ĠWord', 'Ġand', 'Ġwrite', 'Ġin', 'Ġit', 'Ġsumm', 'er', 'iz', 'ation', 'Ġabout', 'Ġp', 'al', 'ast', 'ine']
Sample text tokens ids
[315, 877, 275, 809, 292, 413, 2087, 277, 368, 321, 411, 283, 298, 417, 460]
Sample text tokens seq
[ 315  877  275  809  292  413 2087  277  368  321  411  283  298  417
  460]


In [34]:
token_embed = tf.keras.layers.Embedding(tokenizer.get_vocab_size(), 4)
token_embeddngs = token_embed(sample_text_tokens_seq)

print("Embedding for the sample text : ", sample_text)
print(token_embeddngs)

Embedding for the sample text :  Open Word and write in it summerization about palastine
tf.Tensor(
[[ 2.43981965e-02  4.32909243e-02  1.39628313e-02  2.24301331e-02]
 [-2.39945948e-04 -1.88515782e-02 -3.20256725e-02  3.17372121e-02]
 [-3.75508666e-02  1.21075027e-02  4.71428744e-02  5.23849577e-03]
 [-4.38168310e-02  4.13910635e-02 -1.02059841e-02 -4.02729139e-02]
 [ 1.95409767e-02  3.10423709e-02 -2.72880923e-02 -1.81930885e-02]
 [ 1.01829991e-02 -3.73161957e-03 -1.30059011e-02  1.32005326e-02]
 [-4.11026850e-02  4.00361083e-02 -2.08054855e-03  3.19932587e-02]
 [-2.25302931e-02 -7.44291395e-03  1.42590888e-02  1.14674680e-02]
 [ 2.62844898e-02  7.29151815e-03  1.14726909e-02 -4.38278913e-03]
 [ 6.42897934e-03 -1.15264282e-02 -7.52918422e-05  1.69273652e-02]
 [-1.20065585e-02 -4.50087786e-02  1.79853328e-02 -1.51856914e-02]
 [ 1.46335848e-02  1.29268058e-02 -4.37643789e-02 -5.95689937e-03]
 [ 9.14343446e-03  1.47856139e-02 -3.95861492e-02  3.09747718e-02]
 [ 3.10674943e-02 -6.25617430

In [35]:
max_sequnce_length = 256
positional_embedding = tf.keras.layers.Embedding(max_sequnce_length, 4)

position_index = tf.range(len(sample_text_tokens_seq))
print(position_index)

tf.Tensor([ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14], shape=(15,), dtype=int32)


In [36]:
positional_embeddings = positional_embedding(position_index)
print("Position embeddings for the input sequence \n", positional_embeddings)

Position embeddings for the input sequence 
 tf.Tensor(
[[ 0.04226061  0.01109152 -0.00798912  0.03700555]
 [ 0.01359606 -0.00651092  0.02175531 -0.01006337]
 [ 0.00189712 -0.01514889 -0.00655063 -0.00149872]
 [ 0.00625813  0.02797654 -0.03521298  0.01616326]
 [ 0.01092789  0.00378226  0.02858016 -0.01607525]
 [-0.04170746 -0.04275035  0.04369533  0.01550922]
 [ 0.04611839  0.0442868  -0.03716939 -0.02281   ]
 [ 0.0088494  -0.01167253 -0.03126861 -0.04702674]
 [-0.03758702  0.03927287 -0.02767576  0.04231388]
 [ 0.03472871  0.01334918 -0.01548441  0.00360351]
 [-0.01614972 -0.0235409   0.02180401 -0.02056655]
 [ 0.02121658  0.03930049  0.01794142  0.01965192]
 [-0.02625407  0.03588239  0.04499065  0.03089705]
 [-0.04133794  0.0475612  -0.01136193 -0.01963928]
 [ 0.02006057 -0.01029266 -0.02462491  0.01797101]], shape=(15, 4), dtype=float32)


In [37]:
input = token_embeddngs + positional_embeddings
print("Input to the initial encoder block : \n", input)

Input to the initial encoder block : 
 tf.Tensor(
[[ 0.06665881  0.05438244  0.00597371  0.05943569]
 [ 0.01335612 -0.0253625  -0.01027036  0.02167384]
 [-0.03565375 -0.00304139  0.04059224  0.00373977]
 [-0.0375587   0.0693676  -0.04541897 -0.02410966]
 [ 0.03046887  0.03482463  0.00129206 -0.03426834]
 [-0.03152446 -0.04648197  0.03068943  0.02870975]
 [ 0.0050157   0.08432291 -0.03924993  0.00918326]
 [-0.01368089 -0.01911545 -0.01700952 -0.03555927]
 [-0.01130253  0.04656439 -0.01620307  0.03793109]
 [ 0.04115769  0.00182275 -0.01555971  0.02053087]
 [-0.02815628 -0.06854968  0.03978934 -0.03575224]
 [ 0.03585017  0.0522273  -0.02582296  0.01369502]
 [-0.01711064  0.050668    0.0054045   0.06187182]
 [-0.01027045  0.04130503 -0.01571164 -0.04673757]
 [-0.01429326 -0.00107246  0.00296687  0.03254782]], shape=(15, 4), dtype=float32)


Batch input preprocessing steps

In [38]:
# Batching
input_batch = [
    'Open Word and save it in "Home directory" as "my_word_file"',
    'Connect the wifi to "Aizen-sama" network',
    'Search about palastine new today and give me a summary about it'
]


In [46]:
# encode the bache
input_batch_encodeing = tokenizer.encode_batch(input_batch)

# input sequences
input_seqs = []

input_seqs.append(input_batch_encodeing[0].ids)
input_seqs.append(input_batch_encodeing[1].ids)
input_seqs.append(input_batch_encodeing[2].ids)

print("Vectorized inputs : \n")
print(input_seqs)

# padding the inputs to be in the same length
padded_input_seqs = tf.keras.preprocessing.sequence.pad_sequences(input_seqs, padding="post")
print("input to the encoder is : ")
print(padded_input_seqs.shape)
print(padded_input_seqs)

Vectorized inputs : 

[[315, 877, 275, 1640, 413, 292, 225, 6, 1933, 414, 6, 862, 225, 6, 81, 93, 67, 91, 835, 67, 74, 514, 6], [428, 272, 304, 334, 77, 295, 225, 6, 37, 2097, 17, 87, 380, 69, 6, 899], [318, 411, 283, 298, 417, 460, 384, 295, 72, 917, 275, 484, 765, 1297, 261, 358, 411, 413]]
input to the encoder is : 
(3, 23)
[[ 315  877  275 1640  413  292  225    6 1933  414    6  862  225    6
    81   93   67   91  835   67   74  514    6]
 [ 428  272  304  334   77  295  225    6   37 2097   17   87  380   69
     6  899    0    0    0    0    0    0    0]
 [ 318  411  283  298  417  460  384  295   72  917  275  484  765 1297
   261  358  411  413    0    0    0    0    0]]


In [47]:
encoder_mask = tf.cast(tf.math.not_equal(padded_input_seqs, 0), tf.float32)
print('padded input : ')
print(padded_input_seqs, '\n')
print("Encoder mask : ")
print(encoder_mask)

padded input : 
[[ 315  877  275 1640  413  292  225    6 1933  414    6  862  225    6
    81   93   67   91  835   67   74  514    6]
 [ 428  272  304  334   77  295  225    6   37 2097   17   87  380   69
     6  899    0    0    0    0    0    0    0]
 [ 318  411  283  298  417  460  384  295   72  917  275  484  765 1297
   261  358  411  413    0    0    0    0    0]] 

Encoder mask : 
tf.Tensor(
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]], shape=(3, 23), dtype=float32)


In [48]:
# expanded dimenstion of the mask
encoder_mask = encoder_mask[:, tf.newaxis, tf.newaxis, :]
encoder_mask

<tf.Tensor: shape=(3, 1, 1, 23), dtype=float32, numpy=
array([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1.]]],


       [[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 0., 0., 0., 0., 0., 0., 0.]]],


       [[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 0., 0., 0., 0., 0.]]]], dtype=float32)>

# Encoder 